In [2]:
import networkx as nx
import urllib.parse
from extend_metalink import *
from SameAsEqGraph import *
from rfc3987 import  parse

ModuleNotFoundError: No module named 'hdt'

In [12]:
def obtaing_ee_graph(graph):
    #step 1: make an authority map to node
    authority_map = {}
    variance_map = {}

    for n in graph.nodes:
        d = parse(n, 'IRI') # ’IRI_reference’ rfc3987

        if d['authority'] in authority_map.keys():
            authority_map[d['authority']].append(n)
        else:
            authority_map[d['authority']] = n
        
        # step 2: construct a dictionary of each node against
        # decoding and add to the list
        variance_map[n] = set()
        
        uq = urllib.parse.unquote(n)
        if uq != n:
            variance_map[n].add(uq)

        # get an encoding of the current iri
        prefix, sign, name = get_name(n)
        quote_name = urllib.parse.quote(name)
        new_iri = prefix + quote_name

        if new_iri != n:
            variance_map[n].add(new_iri)
    
    encoding_equality_graph = nx.Graph()
    for iri_with_same_authority in authority_map.values():
        for i in iri_with_same_authority:
            for j in iri_with_same_authority:
                if i != j: # avoid reflexive edges
                    if i in variance_map[j] or j in variance_map[i]: # test if i is the same as any of j's variances
                        encoding_equality_graph.add_edge(i,j)
    
    print ('original graph has ', graph.number_of_nodes(), ' nodes')
    print ('original graph has ', graph.number_of_edges(), ' edges')
    print ('there are ', encoding_equality_graph.number_of_nodes() , ' nodes in the equivalence graph')
    print ('there are ', encoding_equality_graph.number_of_edges() , ' edges in the equivalence graph')

    return encoding_equality_graph

In [11]:
def export_ee_graph_edges(file_name, graph):
    with open(file_name, 'w') as output:
        for (l,r) in graph.edges():
            line = '<' + 1 + '>'
            line += '<' + my_encoding_equivalence + '>'
            if r[0] == '"':
                if "^^" in r:
                    line += '' + r + '.\n'
                else:
                    line += '' + r + "^^<http://www.w3.org/2001/XMLSchema#string>" + ' .\n'
            else:
                line += '<' + r + '>'
            output.write(str(line))